In [1]:
import numpy as np
from numpy.linalg import cholesky

In [12]:
def cov_mat(f, t_vec):
    """
    Produce the covariance matrix for the given autocorreltion function

    :param f: The autocorrelation function of the noise (with fitted parameters)
    :param t_vec: The time series vector
    :return: The covariance matrix for the given autocorrelation function
    """

    return [[f(abs(t1 - t2)) for t2 in t_vec] for t1 in t_vec]

In [10]:
def multivariate_normal(vec, t_vec, f):
    """
    Create a noise sampled from multivariate gaussian.
    When the autocorrelation function of the noise is known.

    :param vec: noise will be added to this vector
    :param t_vec: The time series vector (noise is correlated in time)
    :param f: The autocorrelation function of the noise (with fitted parameters)
    :return: vec with noise
    """

    # Generate noise sampled from standard normal pdf
    noise = np.random.normal(size=len(vec))

    # Decompose covariance matrix
    # And return noise with given correlation
    covmat = cov_mat(f=f, t_vec=t_vec)
    return np.dot(cholesky(covmat), noise)

In [6]:
from neuron import h, gui

def exp_model(Ra=157.3621, gpas=0.000403860792, cm=7.849480, dt=0.1):
    # -- Biophysics --
    # Sec parameters and conductance
    for sec in h.allsec():
        sec.Ra = Ra  # Ra is a parameter to infer
        sec.cm = cm   # parameter optimisation algorithm found this
        sec.v = 0

        sec.insert('pas')
        sec.g_pas = gpas  # gpas is a parameter to infer
        sec.e_pas = 0

    # Print information
    #h.psection()

    # Stimulus
    stim1 = h.IClamp(h.soma(0.01))
    stim1.delay = 200
    stim1.amp = 0.5
    stim1.dur = 2.9

    stim2 = h.IClamp(h.soma(0.01))
    stim2.delay = 503
    stim2.amp = 0.01
    stim2.dur = 599.9

    # Run simulation ->
    # Set up recording Vectors
    v_vec = h.Vector()  # Membrane potential vector
    t_vec = h.Vector()  # Time stamp vector
    v_vec.record(h.soma(0.5)._ref_v)
    t_vec.record(h._ref_t)

    # Simulation duration and RUN
    h.tstop = 1200  # Simulation end
    h.dt = dt  # Time step (iteration)
    h.steps_per_ms = 1 / dt
    h.v_init = 0
    h.finitialize(h.v_init)

    h.init()
    h.run()

    t = t_vec.to_python()
    v = v_vec.to_python()

    return t, v


# --- Load NEURON morphology
h('load_file("/Users/Dani/TDK/parameter_estim/exp/morphology_131117-C2.hoc")')
# Set the appropriate "nseg"
for sec in h.allsec():
    sec.Ra = 160
h('forall {nseg = int((L/(0.1*lambda_f(100))+.9)/2)*2 + 1}')  # If Ra_max = 105 dend.nseg = 21 and soma.nseg = 1

1

In [7]:
t, v = exp_model()

In [18]:
from math import exp
def autocorrelation_function(x):
    D = 4.55090676e-01
    lamb = 2.82009826e-02
    A = 5.89036519e-03
    mu = 1.28340772e+02
    sig = 7.38233398e+01
    return D*lamb*exp(-lamb*abs(x)) - A*exp(-(abs(x)-mu)**2/(2*sig**2))

In [19]:
v_noised = multivariate_normal(v, t, autocorrelation_function)

LinAlgError: Matrix is not positive definite